# Chapter 1  Exploratory data analysis

Anecdotal evidence usually fails, because:
- Small number of observations
- Selection bias
- Confirmation bias
- Inaccuracy

To address the limitations of anecdotes, we will use the tools of statistics, which include:
- Data collection
  - large data
  - valid data
- Descriptive statistics
  - summary statistics
  - visualization
- Exploratory data analysis
  - patterns
  - differences
  - inconsistencies & limitations
- Estimation
  - sample, population
- Hypothesis testing
  - group
  

## DataFrames

In [12]:
import nsfg
df = nsfg.ReadFemPreg()
df.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
0,1,1,NaN,NaN,NaN,NaN,6,NaN,1,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125
1,1,2,NaN,NaN,NaN,NaN,6,NaN,1,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750
2,2,1,NaN,NaN,NaN,NaN,5,NaN,3,5,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,9.1250
3,2,2,NaN,NaN,NaN,NaN,6,NaN,1,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,7.0000
4,2,3,NaN,NaN,NaN,NaN,6,NaN,1,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,6.1875


In [7]:
df.columns

Index([         u'caseid',        u'pregordr',       u'howpreg_n',
             u'howpreg_p',        u'moscurrp',        u'nowprgdk',
              u'pregend1',        u'pregend2',        u'nbrnaliv',
              u'multbrth',
       ...
            u'laborfor_i',      u'religion_i',         u'metro_i',
               u'basewgt', u'adj_mod_basewgt',        u'finalwgt',
                u'secu_p',            u'sest',         u'cmintvw',
           u'totalwgt_lb'],
      dtype='object', length=244)

In [11]:
pregordr = df['pregordr']
pregordr.head()

0    1
1    2
2    1
3    2
4    3
Name: pregordr, dtype: int64

## Glossary
- **anecdotal evidence**: Evidence, often personal, that is collected casually rather than by a well-designed study.
- **population**: A group we are interested in studying. “Population” often refers to a group of people, but the term is used for other subjects, too.
- **cross-sectional study**: A study that collects data about a population at a particular point in time.
- **cycle**: In a repeated cross-sectional study, each repetition of the study is called a cycle.
- **longitudinal study**:: A study that follows a population over time, collecting data from the same group repeatedly.
- **record**: In a dataset, a collection of information about a single person or other subject.
- **respondenv**: A person who responds to a survey.
- **sample**: The subset of a population used to collect data.
- **representative**: A sample is representative if every member of the population has the same chance of being in the sample.
- **oversampling**: The technique of increasing the representation of a sub-population in order to avoid errors due to small sample sizes.
- **raw data**: Values collected and recorded with little or no checking, calculation or interpretation.
- **recode**: A value that is generated by calculation and other logic applied to raw data.
- **data cleaning**: Processes that include validating data, identifying errors, translating between data types and representations, etc.

## Public Data
Governments are good sources because data from public research is often freely available.
Good places to start include:
- http://www.data.gov/
- http://www.science.gov/
- http://data.gov.uk/

Two of book auther's favorite data sets are:
- General Social Survey http://www3.norc.org/gss+website/
- European Social Survey  http://www.europeansocialsurvey.org/

